# Using `keras`

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=ImportWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ResourceWarning)

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

## Access to backend

The backend provides a consistent interface for accessing useful data manipulaiton functions, similar to numpy. It uses whaterver engine is powerinng `keras` - in our case, it uses `TensorFlow`, but it can also use `Theano` and `CNTK` - in each case, the API is the same. 

See [docs](https://keras.io/backend/)

In [ ]:
import keras.backend as K

In [ ]:
m = 5
n = 3
p = 4

a = K.random_normal([m, n], mean=0, stddev=1)
b = K.random_normal([n, p], mean=0, stddev=1)
c = K.dot(a, b)

K.eval(c)

## Linear regression

In [ ]:
np.random.seed(123)

In [ ]:
N = 10
W_true = 2
b_true = 1
eps = np.random.normal(0, 1, (N, 1))
X_obs = np.arange(N).reshape((N, 1))
y_obs = np.reshape(W_true * X_obs + b_true + eps, (-1, 1))

In [ ]:
X_obs.shape

### Build model

In [ ]:
model = Sequential()
model.add(Dense(units=1, input_dim=1, activation=None))

### Compile model with optimizer, loss function and evaluaiton metrics

In [ ]:
model.compile(
    optimizer='rmsprop', 
    loss='mse', 
    metrics=['mse']) 

### Fit model

In [ ]:
history = model.fit(
    X_obs, 
    y_obs,
    batch_size=N, 
    epochs=1000,
    verbose=0)

### Visualize progress

In [ ]:
plt.figure(figsize=(12,4))
hist = history.history
plt.subplot(121)
plt.plot(hist['loss'])
plt.title('loss')
plt.subplot(122)
plt.plot(hist['mean_squared_error'])
plt.title('metric')
pass

### Get fitted parameters

In [ ]:
model.get_weights()

## Logistic Regression

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [ ]:
df = pd.get_dummies(df, columns=['rank'], drop_first=True)
df.head()

In [ ]:
N = df.shape[0]

In [ ]:
X = df.iloc[:, 1:].values
X = (X - X.mean(0))/X.std(0)
y = df.iloc[:, 0:1].values

In [ ]:
X.shape

In [ ]:
y.shape

### Build model

In [ ]:
model = Sequential()
model.add(Dense(1, input_dim=X.shape[1], activation='sigmoid'))

### Compile model

In [ ]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

### Fit model

In [ ]:
history = model.fit(
    x=X,
    y=y,
    batch_size=64, 
    epochs=100, 
    verbose=0)

### Visualize progress

In [ ]:
plt.figure(figsize=(12,4))
hist = history.history
plt.subplot(121)
plt.plot(hist['loss'])
plt.title('loss')
plt.subplot(122)
plt.plot(hist['acc'])
plt.title('metric')
pass

In [ ]:
model.get_weights()

## Regularization and model evaluation

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Add regularization to model

In [ ]:
from keras.regularizers import l2

In [ ]:
reg = l2()
model = Sequential()
model.add(
    layer=Dense(1, 
                input_dim=X.shape[1], 
                activation='sigmoid', 
                kernel_regularizer=reg))

In [ ]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=64, 
    epochs=300, 
    verbose=0,
    validation_data = (X_test, y_test))

In [ ]:
history.history.keys()

In [ ]:
plt.figure(figsize=(12,4))
hist = history.history
plt.subplot(121)
plt.plot(hist['loss'])
plt.plot(hist['val_loss'], c='red')
plt.title('loss')
plt.subplot(122)
plt.plot(hist['acc'])
plt.plot(hist['val_acc'], c='red')
plt.title('accuracy')
pass

In [ ]:
loss, acc = model.evaluate(X_test, y_test)

In [ ]:
loss, acc

## Saving and loading models

### Save model

In [ ]:
s = model.to_json()
with open('logistic_model.json', 'w') as f:
    f.write(s) 

### Save weights

In [ ]:
model.save_weights('logistic_wts.h5') 

### Load model and weights

In [ ]:
from keras.models import model_from_json

In [ ]:
with open('logistic_model.json') as f:
    s = f.read()
    model_1 = model_from_json(s)
model_1.load_weights('logistic_wts.h5')